In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/126471648269214406-software-engineer-chrome-accessibility?q=%22software%20engineer%22")
page_data = loader.load().pop().page_content


In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### Scraped text from the Website:
    {page_data}
    ### Instruction:
    The scraped text is from a career's paeg of a website.
    Your job is to extract the job openings and return them in JSON format containing
    the following keys: 'role', 'experience', 'skills' and 'description' .
    Only return valid JSON.
    ### Valid JSON (No Preamble):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
[
  {
    "role": "Software Engineer, Chrome Accessibility",
    "experience": "Mid",
    "skills": [
      "Data structures/algorithms",
      "Software development",
      "C++",
      "JavaScript",
      "Web Applications",
      "Chrome",
      "Chrome OS"
    ],
    "description": "Work closely with User Experience (UX) designers, Product Managers, and other engineers to define, design, and implement accessibility and immersive reading features."
  },
  {
    "role": "Software Engineer, Multimedia, Kernel and Firmware, Silicon",
    "experience": null,
    "skills": null,
    "description": "No description available"
  },
  {
    "role": "Software Engineer, PhD, Early Career Campus, Machine Learning, Systems and Cloud AI, GPU Performance, 2025 Start",
    "experience": "Early Career",
    "skills": [
      "Machine Learning",
      "Systems and Cloud AI",
      "GPU Performance"
    ],
    "description": "No description available"
  },
  {
    "role": "Software Engineer, g

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_parser.parse(res.content)


In [ ]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

In [ ]:
job = json_res
job['skills']

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)